# Redes neuronales: TabNet

## 1.- Descripción

Este algoritmo se basa en una arquitectura de red neuronal profunda diseñada específicamente para manejar datos tabulares

In [ ]:
https://medium.com/%40kdk199604/tabnet-a-deep-learning-breakthrough-for-tabular-data-bcd39c47a81c

https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/tabnet-train?hl=es-419


## 2.- Implementación


## 3.- Evaluación y optimización


## 4.- Conclusión